In [ ]:
import pathlib
import random
import json
import tempfile

import sys
sys.path.append('../src')
from mediatools import ffmpeg
import mediatools

In [ ]:
fn = '/mnt/MoStorage/gopro/GoPro 2024-01-01/GX012082.MP4'
gopro_vf = mediatools.VideoFile(fn)
music_fn = list(pathlib.Path('music_vids/').rglob('*.mp4'))
music_vfs = [mediatools.VideoFile(f) for f in music_fn]

In [ ]:
#for vf in music_vfs:
#    print(vf.read_metadata())
gopro_vf.probe().file_bitrate

In [ ]:
all_vids = list(pathlib.Path('music_vids/').rglob('*.mp4'))
#all_vids = list(pathlib.Path('/mnt/HDDStorage/').rglob('*.mp4'))
vids = all_vids
#vids = list(random.sample(all_vids, k=5))

print(f'{len(all_vids)=}, {len(vids)=}')
compilation = ffmpeg.create_montage(video_files=vids, output_filename='compilation_deleteme_lee.mp4', clip_ratio=10, clip_duration=2.0, verbose=False)

In [ ]:
class VideoFileAlreadyCompressed(Exception):
    pass

def compress_video_by_bitrate(
    vf: mediatools.VideoFile,
    new_fpath: pathlib.Path,
    target_av_bitrate: int,
    vcodec: str = 'libx264',
    overwrite: bool = True,
) -> mediatools.ffmpeg.FFMPEGResult:
    input_info = mediatools.ffmpeg.probe(vf.fpath)
    if input_info.file_bitrate <= target_av_bitrate:
        raise VideoFileAlreadyCompressed(f'\tinput bitrate {input_info.file_bitrate/1000:5.1f} kbps <= target '
                f'({target_av_bitrate/1000:5.1f} kbps). skipping compression.')

    with tempfile.TemporaryDirectory() as tmpdir:
        tmp_dir = pathlib.Path(tmpdir)
        tmp_log_dir = tmp_dir / 'logs/'
        
        # pass 1: ffmpeg -i "your_source_video.mp4" -c:v libx264 -b:v 2000k -pass 1 -an -f mp4 /dev/null
        # pass 2: ffmpeg -i "your_source_video.mp4" -c:v libx264 -b:v 2000k -pass 2 -c:a copy "output_at_2000k.mp4"
        
        result1 = mediatools.ffmpeg.FFMPEG(
            inputs = [mediatools.ffinput(vf.fpath)],
            outputs = [
                mediatools.ffoutput(
                    'NUL',
                    c_v=vcodec,
                    b_v=target_av_bitrate,
                    an=True,
                    f='mp4',
                    y=overwrite,
                ),
            ],
            pass_num=1,
            passlogfile=str(tmp_log_dir),
        ).run()

        result2 = mediatools.ffmpeg.FFMPEG(
            inputs = [mediatools.ffinput(vf.fpath)],
            outputs = [
                mediatools.ffoutput(
                    new_fpath,
                    c_v=vcodec,
                    b_v=target_av_bitrate,
                    c_a='copy',
                )
            ],
            passlogfile=str(tmp_log_dir),
            pass_num=2,
        ).run()
        return result2
    
mediatools.ffmpeg.compress_video_by_bitrate(
    gopro_vf.fpath,
    pathlib.Path('compressed_gopro.mp4'),
    target_av_bitrate=2_500_000,
)

In [ ]:
import json
ffmpeg.probe('/mnt/MoStorage/gopro/GoPro 2024-01-01/GX012082.MP4').tags

In [ ]:
# Example 1: Basic metadata modification using FFOutput
from mediatools.video.ffmpeg import FFMPEG, FFInput, FFOutput

# Change metadata for a video file
input_file = FFInput('/mnt/MoStorage/gopro/GoPro 2024-01-01/GX012082.MP4')
output_file = FFOutput(
    'output_with_metadata.mp4',
    vcodec='copy',  # Copy video without re-encoding
    acodec='copy',  # Copy audio without re-encoding
    metadata={
        'title': 'My GoPro Video',
        'artist': 'John Doe',
        'comment': 'Recorded with GoPro',
        'album': 'Adventure Videos 2024',
        'date': '2024-01-01'
    },
    overwrite=True
)

# Build the command to see what it looks like
cmd = FFMPEG(inputs=[input_file], outputs=[output_file])
print("Command:", cmd.get_command())